In [80]:
import math
import pickle
import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec

from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_absolute_error, r2_score

In [81]:
# Read the file in which the values are saved
appa2 = pd.read_csv("../../exports/appa2_shrinked.csv")
appa2 = appa2.drop('Unnamed: 0', axis=1)

In [82]:
#selecting all the values we need to train and test the model
appa2 = appa2[[
    'LaFeO3_1', 'LaFeO3_2', 
    'STN_1', 'STN_2', 
    'LaFeO3_1_heatR', 'LaFeO3_2_heatR',
    'STN_1_heatR', 'STN_2_heatR', 
    "LaFeO3_1_Age", "LaFeO3_2_Age",
    "STN_1_Age","STN_2_Age",
    'Temperature', 'Relative_Humidity', 'Pressure', 'VOC', 
    'Wind_Speed', 
    'PM10', 'CO', 'NO2'
]]

# Random Forest

In [83]:
# Select the features
cols = [
    'LaFeO3_1', 'LaFeO3_2', 
    'STN_1', 'STN_2', 
    'LaFeO3_1_heatR', 'LaFeO3_2_heatR',
    'STN_1_heatR', 'STN_2_heatR', 
    "LaFeO3_1_Age", "LaFeO3_2_Age",
    "STN_1_Age","STN_2_Age",
    'Temperature', 'Relative_Humidity', 'Pressure', 'VOC', 
    'Wind_Speed'
]

# Select the target
outs = ['PM10', 'CO', 'NO2']


In [74]:
# # Split the values in training and test
# vsplit = round(len(appa2) * 0.2)

# X_train = appa2.iloc[vsplit:][cols]
# X_test = appa2.iloc[:vsplit][cols]

# Y_train = appa2.iloc[vsplit:][outs]
# Y_test = appa2.iloc[:vsplit][outs]

In [84]:
features = appa2[cols]
target= appa2[outs]

features.columns, target.columns

(Index(['LaFeO3_1', 'LaFeO3_2', 'STN_1', 'STN_2', 'LaFeO3_1_heatR',
        'LaFeO3_2_heatR', 'STN_1_heatR', 'STN_2_heatR', 'LaFeO3_1_Age',
        'LaFeO3_2_Age', 'STN_1_Age', 'STN_2_Age', 'Temperature',
        'Relative_Humidity', 'Pressure', 'VOC', 'Wind_Speed'],
       dtype='object'),
 Index(['PM10', 'CO', 'NO2'], dtype='object'))

In [ ]:
kf = RepeatedKFold(n_splits = 10, n_repeats = 1)

for train_index, test_index in kf.split(features.values):
    X_train, X_test = features[train_index], features[test_index]
    Y_train, Y_test = target[train_index], target[test_index]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle = False)

In [ ]:
working_X_train, working_X_test, working_Y_train, working_Y_test = X_train, X_test, Y_train, Y_test

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start= 1000, stop= 2000, num=10)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [4, 5, 6]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [85]:
# Choose the type of model
model = RandomForestRegressor()

In [86]:
model_ris = cross_validate(model, features, target, cv = 10, n_jobs= -1)

In [87]:
model_ris.keys()

model_ris["test_score"]

array([ 0.02287442, -0.20037839, -0.05824684, -0.51274001, -0.79642106,
        0.02923268, -0.64532628, -0.5165638 , -0.35324025, -0.67153109])

In [ ]:
# Perform the RandomForestRegression with a Random Search on hyperparameters
rf_random = RandomizedSearchCV(estimator=model, param_distributions=random_grid, n_iter=100, cv=3, verbose=2,
                               random_state=42, n_jobs=-1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

In [ ]:
# Create a Dataframe with the results of the model and then saving them into a .csv file
ris = pd.DataFrame(rf_random.cv_results_)
ris.to_csv('results/results_weather2_shrinked_shuffle(1).csv')

In [ ]:
# Save the model itself in a .sav file
pickle.dump(rf_random, open('models/model_weather2_shrinked_shuffle(1).sav', 'wb'))

## Results

In [ ]:
rf_random = pickle.load(open("models/model_weather2_shrinked_shuffle(1).sav", "rb"))
rf_random

In [ ]:
def percentage(mean_abs_err, describer):
    perc = mean_abs_err / (describer.max() - describer.min())
    return perc * 100

In [ ]:
print("PM10: " + str(percentage(mean_absolute_error(Y_test.PM10 , rf_random.predict(X_test)[: ,0]), Y_test.PM10) ))
print("CO: " + str(percentage(mean_absolute_error(Y_test.CO , rf_random.predict(X_test)[: ,1]), Y_test.CO)))
print("NO2: " + str(percentage(mean_absolute_error(Y_test.NO2 , rf_random.predict(X_test)[: ,2]), Y_test.NO2)))
print("Total: \n" + str(mean_absolute_error(Y_test, rf_random.predict(X_test))))

In [ ]:
rf_random.predict(X_test)[:, 0]

In [ ]:
# RSS (Residual Sum of Squares) --> it estimates the variance in the residuals
def residual_sum(real, predict):
    rss = 0
    for index in range(len(real)):
        rss += (real[index] - predict[index]) ** 2 
    return rss

# TSS (Total Sum of Squares) --> the squared differences between the observed dependent variable and its mean
def total_sum_square(real):
    tss = 0
    meanR = real.mean()
    for index in range(len(real)):
        tss += (real[index] - meanR) ** 2 
    return tss

# R^2 Coefficient of determination --> Statistical maeasure that represents the proportion of the variance for a dependent variable that's explained by an indipendent variable or variables in a regression model
def r_squared(real, predict):
    value = residual_sum(real, predict)/total_sum_square(real)
    return 1 - value

In [ ]:
print(f"PM10: {r2_score(Y_test.PM10, rf_random.predict(X_test)[:, 0])}" ) 
print(f"CO: {r2_score(Y_test.CO, rf_random.predict(X_test)[:, 1])}")
print(f"NO2: {r2_score(Y_test.NO2, rf_random.predict(X_test)[:, 2])}")

## Hyperparameters Plots

In [ ]:
ris = pd.read_csv("results/results_weather2_shrinked_shuffle(1).csv")

In [ ]:
# Plot the hyper-parameters in relation of mean test score
# Values are plotted through dots. The brighter the dot, the more times that value has been chosen by the models.
# On the X axis, the closer the values are to 0 the better they are

h_params = ['param_n_estimators', 'param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', ]
fig = plt.figure(figsize=(25, 20))
fig.suptitle('Hyper Parameters')
outer = gridspec.GridSpec(3, 2, wspace=0.2, hspace=0.2)
for index, h_param in enumerate(h_params):
    ax = plt.Subplot(fig, outer[index])
    ax.scatter(ris['mean_test_score'], ris[h_param], color='red', alpha=0.4, )
    ax.set_ylabel(h_param)
    ax.set_xlabel('mean_test_score')
    # ax.set_ylim(105, 110)
    fig.add_subplot(ax)
fig.show()

## Feature Importance

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot()
ax.barh(X_test.columns, rf_random.best_estimator_.feature_importances_)
plt.show()

In [ ]:
#Printing the charts to undersand better our predictions

fig,((ax1,ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(20, 20))

ax1.scatter(Y_test.PM10, rf_random.predict(X_test)[:, 0], label = "PM10")
ax2.scatter(Y_test.CO, rf_random.predict(X_test)[:, 1], label = "CO")
ax3.scatter(Y_test.NO2, rf_random.predict(X_test)[:, 2], label = "NO2")

ax1.set_xlim(0, 45)
ax1.set_ylim(0, 45)
ax2.set_xlim(0, 1.5)
ax2.set_ylim(0, 1.5)
ax3.set_xlim(0.5, 90)
ax3.set_ylim(0.5, 90)

ax1.set_title('PM10')
ax1.set_xlabel('Real data')
ax1.set_ylabel('Prediciton')
ax2.set_title('CO')
ax2.set_xlabel('Real data')
ax2.set_ylabel('Prediciton')
ax3.set_title('NO2')
ax3.set_xlabel('Real Data')
ax3.set_ylabel('Prediction')

ax1.legend()
ax2.legend()
ax3.legend()

plt.show()

In [ ]:
gases = Y_test.columns

fig = plt.figure(figsize=(15, 15))
fig.suptitle('Gases')
outer = gridspec.GridSpec(2, 2, wspace=0.2, hspace=0.2)
for index, gas in enumerate(gases):
    ax = plt.Subplot(fig, outer[index])
    ax.scatter(Y_test[gas], rf_random.predict(X_test)[:,index], marker=".", alpha=0.2, label=gas)
    ax.set_xlabel(f"Real")
    ax.set_ylabel(f"Predict")
    ax.set_xlim(0, Y_test[gas].max()*9/8)
    ax.set_ylim(0, Y_test[gas].max()*9/8)
    ax.plot([-100, Y_test[gas].max()*10], [-100,  Y_test[gas].max()*10], c="k", alpha=1)
    ax.legend()
    fig.add_subplot(ax)
fig.show()

In [ ]:
appa2.CO.describe()

In [ ]:
appa2.CO.value_counts()

In [ ]:
Y_train.CO.value_counts()